In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'

import time

from pddlvm.utilities import tf, tfp, np, plt, tfb, floatf, tffloat, tfint, updateTfFloat,  _0
updateTfFloat ( tf.float32 )
tf.keras.backend.set_floatx('float32')

from pddlvm.poisson1DPDEState import PDE_State
from pddlvm.poisson1DSolver import Solver 
from pddlvm.NeuralNet import NeuralN
from pddlvm.VEModel import VE
tf.random.set_seed(42)

import time

print(tffloat(0.))
tf.config.list_physical_devices()

In [ ]:
'''Create PDEState Object with Cheby Num terms (-1 for degree)'''
omegaLength = 2.0
numFElems   = 60
numFENodes  = numFElems + 1
xFE = tffloat( tf.linspace( 0., omegaLength, numFElems + 1) ) - 1.

#pdeState = PDE_State(xFE, dimK=1, dimF=1, dimU=numFENodes)
pdeState = PDE_State(xFE, dimK=5, dimF=1, dimU=10)

'''Modify pdeState Bijcetor for forcing'''
pdeState.kappaBij   = tfb.Chain( bijectors=[tfb.Shift(tffloat(1.)), tfb.Softplus()] )
pdeState.forcingBij = tfb.SoftClip(low=tffloat(-5.), high=tffloat(5.), hinge_softness=1.)
#pdeState.uBij       = tfb.SoftClip(low=tffloat(-10.), high=tffloat(10.), hinge_softness=1.)
print(pdeState.dimK, pdeState.dimU)

In [ ]:
'''Create solver Object'''
solver = Solver( pdeState, xFE, isNonLinear= True )

'''Modify pdeState.stress-strain for nonlinear Youngs Modulus'''
import types
def stress_strain_nonLinear(self, index, kappa, u_trial):
    #print('custom stress-strain')
    dudx = ( u_trial[index] - u_trial[index+1]) / (self.pdeState.xFE[index] - self.pdeState.xFE[index+1])
    return (tf.keras.activations.sigmoid(tf.math.abs(dudx)) + kappa*5. ) /10.
solver.stress_strain = types.MethodType(stress_strain_nonLinear, solver)


In [ ]:
model = VE( pdeState, solver )

def createNNs():
    dimKFBlBr = model.dimK + model.dimF + model.dimBl + model.dimBr
    nh = 100
    U_KFBlBr  = NeuralN('U_of_KFBlBr', 
                    np.array([dimKFBlBr,
                              nh, nh, nh, nh, 
                              model.dimU])
                    )

    k  = model.dimK
    bl = k  + model.dimBl
    br = bl + model.dimBr
    Z_UF     = NeuralN('Z_of_UF',
                    np.array([pdeState.dimU + model.dimF, 
                              nh, nh, nh, nh,
                                model.dimK + model.dimBl + model.dimBr]),
                    multOutputLayout = {'K': tf.range(0,k),
                                        'Bl':tf.range(k,bl),'Br':tf.range(bl,br)}
                    )

    model.initPriorNetworks(U_KFBlBr=U_KFBlBr, Z_UF=Z_UF)
    model.setModePrior()
    return 

    
USE_UNIFORM_PRIOR = False
if USE_UNIFORM_PRIOR == True:
    model.LatentKappaPrior   = tfp.distributions.Uniform(low=tffloat([-0.5]*pdeState.dimK), high=tffloat([0.5]*pdeState.dimK))#works
    model.LatentForcingPrior = tfp.distributions.Uniform(low=tffloat([5.]*pdeState.dimF), high=tffloat([5.]*pdeState.dimF) )

USE_NORMAL_PRIOR = True
if USE_NORMAL_PRIOR == True:
    model.LatentKappaPrior   = tfp.distributions.Normal(tffloat([0.]*pdeState.dimK), tffloat([1.]*pdeState.dimK))
    model.LatentForcingPrior = tfp.distributions.Uniform(low=tffloat([1.]*pdeState.dimF), high=tffloat([2.]*pdeState.dimF) )#works

model.LatentBlPrior      = tfp.distributions.Uniform(low=tffloat([0.]), high=tffloat([0.]))
model.LatentBrPrior      = tfp.distributions.Uniform(low=tffloat([0.5]), high=tffloat([1.]) )


print(model.LatentForcingPrior.sample())
print(model.LatentBlPrior.sample())
createNNs()


In [ ]:
totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

def computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=True, SAVE=None):
    
    global totalTimeNNSolving
    global totalTimeSolving  
    
    startSolving = time.time()
    meanSamplesU, logVarSamplesU = model.priorModels['U_of_KFBlBr'].Map([samplesK, samplesF, samplesBcl, samplesBcr])
    endSolving = time.time()
    totalTimeNNSolving += endSolving - startSolving

    us = model.reparameterize(meanSamplesU, logVarSamplesU )

    residualsU = tf.constant(0., shape=[1])
    residualsK = tf.constant(0., shape=[1])
    residualsB = tf.constant(0., shape=[1])
    
    if PLOT == True:
        fig, ax = plt.subplots(nrows=1, ncols=5, sharex=True,
                                            figsize=(24, 4))
    
    for i in range(samplesK.shape[0]):

        pdeState.kappa   = samplesK[i,:] 
        pdeState.forcing = samplesF[i,:] 
        pdeState.bdLeft  = samplesBcl[i,0] 
        pdeState.bdRight = samplesBcr[i,0] 

        startSolving = time.time()
        u_true     = solver.solve_tf_bfgs(  )
        u_trueSpec = solver.solve_tf_FESpectral_bfgs( )
        u_trueSpec = pdeState.u_funXFE( u_trueSpec[:,None] )
        endSolving = time.time()
        totalTimeSolving += endSolving - startSolving

        meanU, diagVarU = pdeState.gaussChebyGaussCoeffs(meanSamplesU[i,:], tf.exp(logVarSamplesU[i,:]), xFE )
        std2U = 2.*tf.sqrt(diagVarU)
        
        startSolving = time.time()
        uField = pdeState.u_funXFE(us[i,:][:,None])
        endSolving = time.time()
        totalTimeNNSolving += endSolving - startSolving

        kappaField = pdeState.kappa_fun( xFE )

        U_true_coeffs = pdeState.getChebyCoeffs_fromPoints(xFE, uField, model.dimU - 1)

        ZCoeffReconstMean, ZCoeffReconstlogvar = model.priorModels['Z_of_UF'].Map([ U_true_coeffs[None,:], samplesF[i,:][None,:]])
        kappaCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstMean =  tf.reshape(kappaCoeffReconstMean,[-1])
        kappaCoeffReconstVar  =  tf.exp(tf.reshape(kappaCoeffReconstlogVar,[-1]))
        meanK, diagCovK = pdeState.gaussChebyGaussCoeffs(kappaCoeffReconstMean,kappaCoeffReconstVar, xFE)
        meanKT, varKT = pdeState.unscentedGaussTransf(meanK, diagCovK, pdeState.kappaBij, k=2)
        std2KT = 2*tf.sqrt(varKT)
        print('std2KT = ', std2KT)

        BlReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Bl'], axis=1)
        BrReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Br'], axis=1)

        residualUi = ( tf.linalg.norm(  u_true - meanU ) / tf.linalg.norm( u_true ) )**2.
        residualKi = ( tf.linalg.norm( kappaField - meanKT ) / tf.linalg.norm( kappaField ) )**2.
        residualBi = ( tf.linalg.norm( ( BlReconst - samplesBcl[i,0] ) + ( samplesBcr[i,0] - BrReconst ) ) / tf.linalg.norm( samplesBcl[i,0] + samplesBcr[i,0] ) )**2.

        residualsU = tf.concat([residualsU, residualUi[None]], 0)
        residualsK = tf.concat([residualsK, residualKi[None]], 0)
        residualsB = tf.concat([residualsB, residualBi[None]], 0)

        
        if PLOT == True:
            ax[0].plot( xFE , u_true)
            ax[0].set_xlabel(r'$x$')
            ax[0].set_ylabel(r'$u(x)$')
     
            ax[1].plot(xFE, meanU)
            ax[1].fill_between(xFE, meanU + std2U, meanU - std2U, alpha=0.2)
            ax[1].set_xlabel(r'$x$')
            ax[1].set_ylabel(r'$u(x) \pm 2\sigma$')

            ax[2].plot( xFE, kappaField)
            ax[2].set_xlabel(r'$x$')
            ax[2].set_ylabel(r'$\kappa(x)$')
            ax[2].set_ylim(0.9, 5.)

            
            ax[3].plot(xFE, meanKT)
            ax[3].fill_between(xFE, meanKT + std2KT, meanKT- std2KT, alpha=0.2)
            ax[3].set_xlabel(r'$x$')
            ax[3].set_ylabel(r'$\kappa(x) \pm 2\sigma$')
            ax[3].set_ylim(0.9, 5.)

            ax[4].plot( xFE , tf.squeeze(pdeState.forcing_funXFE()))
            ax[4].set_xlabel(r'$x$')
            ax[4].set_ylabel(r'$f(x)$')


    if PLOT == True:
        plt.tight_layout()
        if SAVE is not None:
            plt.tight_layout()
            extent = ax[0].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig('figs/{}-FESolution.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

            extent = ax[1].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig('figs/{}-NNSolution.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

            extent = ax[2].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig('figs/{}-TrueKappa.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

            extent = ax[3].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig('figs/{}-NNKappa.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

            extent = ax[4].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
            fig.savefig('figs/{}-Forcing.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

            plt.show()

    return residualsU, residualsK, residualsB

In [ ]:
numSample = 10
PLOT = True
sampleKGenerator = model.LatentKappaPrior

samplesK   = sampleKGenerator.sample(sample_shape=(numSample))
samplesF   = model.LatentForcingPrior.sample(sample_shape=(numSample))
samplesBcl = model.LatentBlPrior.sample(sample_shape=(numSample))
samplesBcr = model.LatentBrPrior.sample(sample_shape=(numSample))

totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

residualsU, residualsK, residualsB = computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=PLOT, SAVE="UPDATE_NONLINEAR_2")

print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
eps =1e-2

AllEpsU = []
AllEpsK = []
AllEpsB = []

totalTimeTraining  = 0.

trainModels        = True


intervalELBOsave = 1000

PLOT = True


@tf.function
def train_step_prior(optimizer, iteration):
    with tf.GradientTape(persistent=True) as tape:
        loss = model.compute_nelbo_prior(iteration)
    for priorvars in model.priorModels['trainable_variables']:
        gradients    = tape.gradient(loss, priorvars)
        gradients, globNorm = tf.clip_by_global_norm(gradients, 1.0)
        tf.debugging.check_numerics(gradients[0], "GRAD NOT FINITE")
        optimizer.apply_gradients(zip(gradients, priorvars))
    return loss

if trainModels:
       
    '''Optimization sequence'''
    model.n_sgd      = 1

    num_iterations   = tfint(1_000_000)

    model.epsilon_r.assign( tffloat(eps) )

    lr = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-3, int(num_iterations/10), 0.5, staircase=True, name=None
    )
    optimizer = tf.keras.optimizers.Adam( lr )

    elboAll     = []
    startTraining = time.time()
    for epoch in tf.range(0, num_iterations ):

        start_time = time.time()
        loss = train_step_prior(optimizer, epoch)

        end_time = time.time()

        if epoch % intervalELBOsave == 0:
            #tf.print('epsi_r = ', model.epsilon_r)
            elboAll.append(-loss)
            print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
                .format(epoch, -loss, end_time - start_time))

    endTraining = time.time()
    totalTimeTraining += endTraining - startTraining
    print("DONE TRAINING FOR EPSI", eps)




In [ ]:
'''If you want to load Model Weights'''

saveModelWeights = True
loadModelWeights = False


if saveModelWeights:
    dirname = 'SAVEDModel/'
    os.makedirs(os.path.dirname(dirname), exist_ok=True)
    model.priorModels['U_of_KFBlBr'].NN.compile()
    model.priorModels['Z_of_UF'].NN.compile()
    model.priorModels['U_of_KFBlBr'].NN.save(dirname+'/2U_of_KFBlBr_weights.model')
    model.priorModels['Z_of_UF'].NN.save(dirname+'/2Z_of_UF_weights.model')
    model.priorModels['U_of_KFBlBr'].NN.save_weights(dirname+'/2U_of_KFBlBr_weights.h5')
    model.priorModels['Z_of_UF'].NN.save_weights(dirname+'/2Z_of_UF_weights.h5')
    print('SAVED MODEL WEIGHTS')
    

if loadModelWeights:
    dirname = 'SAVEDModel/'
    model.priorModels['U_of_KFBlBr'].NN = tf.keras.models.load_model(dirname+'/2U_of_KFBlBr_weights.model')
    model.priorModels['Z_of_UF'].NN     = tf.keras.models.load_model(dirname+'/2Z_of_UF_weights.model')

    model.priorModels['U_of_KFBlBr'].NN.load_weights(dirname+'/2U_of_KFBlBr_weights.h5')
    model.priorModels['Z_of_UF'].NN.load_weights(dirname+'/2Z_of_UF_weights.h5')

    model.priorModels['U_of_KFBlBr'].NN.compile()
    model.priorModels['Z_of_UF'].NN.compile()
    # Check its architecture
    model.priorModels['U_of_KFBlBr'].NN.summary()
    model.priorModels['Z_of_UF'].NN.summary()
    print('LOADED MODEL WEIGHTS')



#print(elboAll)
if trainModels:
    elboAll =  np.array(elboAll)
    np.savetxt('ELBOValsModel-{}epsi'.format(eps), elboAll)

In [ ]:
totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

residualsU, residualsK, residualsB = computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=PLOT, SAVE="UPDATE_PostTrainingNonLINEAR")
print( 'totalTimeTraining = ', totalTimeTraining)
print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
numSample  = 5
samplesK   = model.LatentKappaPrior.sample(sample_shape=(numSample)) + 1.
samplesF   = model.LatentForcingPrior.sample(sample_shape=(numSample))
samplesBcl = model.LatentBlPrior.sample(sample_shape=(numSample)) + 0.5
samplesBcr = model.LatentBrPrior.sample(sample_shape=(numSample)) + 0.5

residualsU, residualsK, residualsB = computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=PLOT, SAVE="PostTrainingNonLINEAR")
print( 'totalTimeTraining = ', totalTimeTraining)
print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
elboAll = np.array(elboAll)
np.savetxt('elboALL', elboAll)
plt.plot(range(elboAll.shape[0]),  elboAll )
scale = 1e3
plt.ylim(-scale,scale)
plt.show()


In [ ]:
with open('Times.txt', 'w') as the_file:
    the_file.write('totalTimeTraining = '  + str(totalTimeTraining) + 's'+ '\n' )
    the_file.write('totalTimeSolving = '  + str(totalTimeSolving) + 's'+  '\n' )
    the_file.write('totalTimeNNSolving = ' + str(totalTimeNNSolving) + 's'+  '\n' )